In [2]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [3]:
epochs = 300
batch_size = 32
patience = 10
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [4]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(25, 25),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(1024,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 25, 25, 3)         0         
                                                                 
 rescaling (Rescaling)       (None, 25, 25, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 256)       7168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 256)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 256)         590080    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 4, 256)         0         
 g2D)                                                 

In [5]:
train = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

Found 40825 files belonging to 62 classes.
Using 32660 files for training.
Found 40825 files belonging to 62 classes.
Using 8165 files for validation.


In [6]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/300


1021/1021 [==============================] - 60s 58ms/step - loss: 0.8144 - accuracy: 0.7652 - val_loss: 0.3109 - val_accuracy: 0.9100
Epoch 2/300
1021/1021 [==============================] - 61s 59ms/step - loss: 0.6103 - accuracy: 0.8196 - val_loss: 0.2389 - val_accuracy: 0.9237
Epoch 3/300
1021/1021 [==============================] - 60s 59ms/step - loss: 0.5087 - accuracy: 0.8491 - val_loss: 0.2066 - val_accuracy: 0.9319
Epoch 4/300
1021/1021 [==============================] - 62s 61ms/step - loss: 0.4592 - accuracy: 0.8626 - val_loss: 0.2077 - val_accuracy: 0.9222
Epoch 5/300
1021/1021 [==============================] - 60s 59ms/step - loss: 0.4253 - accuracy: 0.8755 - val_loss: 0.1804 - val_accuracy: 0.9435
Epoch 6/300
1021/1021 [==============================] - 60s 59ms/step - loss: 0.3998 - accuracy: 0.8837 - val_loss: 0.1664 - val_accuracy: 0.9476
Epoch 7/300
1021/1021 [==============================] - 60s 59ms/step - loss: 0.3719 - accuracy: 0.8904 - val_loss: